In [ ]:
# Setup input parameters
from datetime import datetime as dt
dbutils.widgets.text('bg_loadtimestamp', '')
bg_loadtimestamp = dbutils.widgets.get('bg_loadtimestamp')
bg_loadtimestamp_dt = dt.strptime(bg_loadtimestamp, '%Y-%m-%d %H:%M:%S.%f')
if not bg_loadtimestamp:
    bg_loadtimestamp = 'CAST(NULL AS Timestamp)'
else:
    bg_loadtimestamp = f"CAST('{bg_loadtimestamp}' AS Timestamp)"




In [ ]:
# Setup logging
def info(statement_name, target_object_database_name, target_object_schema_name, target_object_name, message):
    log_message('INFO', statement_name, target_object_database_name, target_object_schema_name, target_object_name, message)

def error(statement_name, target_object_database_name, target_object_schema_name, target_object_name, message):
    log_message('ERROR',statement_name, target_object_database_name, target_object_schema_name, target_object_name, message)

def log_message(log_level, statement_name, target_object_database_name, target_object_schema_name, target_object_name, message):
    log_df = spark.createDataFrame([(dt.now(), log_level, 'loader', 'C3-X-DB-DV', '{loadcontrol#loadcontrol#application_name}', '{loadcontrol#loadcontrol#application_environment_name}', bg_loadtimestamp_dt, statement_name, 'RDV_PIT_ORDERDETAIL_Loader', target_object_database_name, target_object_schema_name, target_object_name,  message)], ['log_timestamp', 'log_level', 'execution_unit', 'project_name', 'application_name', 'application_environment_name', 'load_timestamp', 'statement_name', 'task_name', 'target_object_database_name', 'target_object_schema_name', 'target_object_name', 'message'])
    log_df.write.format('delta').mode('append').saveAsTable('`{loadcontrol#loadcontrol#database_name}`.`{loadcontrol#loadcontrol#schema_name}`.`{loadcontrol#loadcontrol#log_table_name}`')
    print(f"{dt.now().strftime('%Y/%m/%d, %H:%M:%S')} - {target_object_name}: {message}")




In [ ]:
# PitLoader: ORDERDETAIL_PIT Loader_1

try:

    operation_metrics_collection = {}
    result_df = spark.sql(f"""
    INSERT
    INTO `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_orderdetail` (
         `bg_loadtimestamp`
        ,`bg_validfromtimestamp`
        ,`pit_hk`
        ,`orderdetail_bg_validfromtimestamp`
        ,`orderdetail_orderdetail_hk`
    )
    SELECT
         {bg_loadtimestamp} AS `bg_loadtimestamp`
        ,`bg_source`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
        ,`bg_source`.`pit_hk` AS `pit_hk`
        ,`bg_source`.`orderdetail_bg_validfromtimestamp` AS `orderdetail_bg_validfromtimestamp`
        ,`bg_source`.`orderdetail_orderdetail_hk` AS `orderdetail_orderdetail_hk`
    FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_orderdetail_delta` AS `bg_source`
    """)
    RowCountInserted = result_df.select("num_inserted_rows").collect()[0][0]
    pandas_df = result_df.toPandas()
    operation_metrics = pandas_df.to_dict(orient='records')
    operation_metrics_collection['inserttarget_{c3-x-db-dv#rawvault#database_name}_{c3-x-db-dv#rawvault#schema_name}_rdv_pit_orderdetail'] = operation_metrics
    info('InsertTarget', '{c3-x-db-dv#rawvault#database_name}', '{c3-x-db-dv#rawvault#schema_name}', 'RDV_PIT_ORDERDETAIL', str(operation_metrics))

except Exception as e:
    error('', '', '', '', str(e))
    raise



In [ ]:
dbutils.notebook.exit(operation_metrics_collection)
